# HATE SPEECH DETECTION

## DATA PREPROCESSING 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
from fastai.nlp import *
from sklearn.linear_model import LogisticRegression

In [4]:
import pandas as pd
import numpy as np

In [8]:
cols = ['class', 'ids', 'date', 'flag', 'user', 'tweet']

In [9]:
data = pd.read_csv('/home/spriyanshu723/prince/sentiment/training.1600000.processed.noemoticon.csv', names=cols, encoding='latin-1')

In [10]:
data.head()

,class,ids,date,flag,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [11]:
data = data.drop(['ids'], axis=1)
data = data.drop(['date'], axis=1)
data = data.drop(['flag'], axis=1)
data = data.drop(['user'], axis=1)

In [12]:
data.head()

,class,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [13]:
data = data[['tweet', 'class']]

In [14]:
data.head()

,tweet,class
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
tweet    1600000 non-null object
class    1600000 non-null int64
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [17]:
data.describe()

,class
count,1.600000e+06
mean,2.000000e+00
std,2.000001e+00
min,0.000000e+00
25%,0.000000e+00
50%,2.000000e+00
75%,4.000000e+00
max,4.000000e+00


In [18]:
data.shape

(1600000, 2)

## RETRIVING TOP WORDS FROM HATE DATASET

In [19]:
data_hate_five = data.loc[data['class'] == 0]
data_hate_five.head()

,tweet,class
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [20]:
data_hate_five.shape

(800000, 2)

In [21]:
data_unhate_five = data.loc[data['class'] == 4]
data_unhate_five.head()

,tweet,class
800000,I LOVE @Health4UandPets u guys r the best!!,4
800001,im meeting up with one of my besties tonight! ...,4
800002,"@DaRealSunisaKim Thanks for the Twitter add, S...",4
800003,Being sick can be really cheap when it hurts t...,4
800004,@LovesBrooklyn2 he has that effect on everyone,4


In [22]:
data_unhate_five.shape

(800000, 2)

In [23]:
with open('a_hate_five', 'wb') as fp:
    pickle.dump(data_hate_five, fp)

In [24]:
with open('a_unhate_five', 'wb') as fp:
    pickle.dump(data_unhate_five, fp)

In [25]:
hate_df = data_hate_five['tweet']
hate_df.head()

0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
1    is upset that he can't update his Facebook by ...
2    @Kenichan I dived many times for the ball. Man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: tweet, dtype: object

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
one = TfidfVectorizer(ngram_range=(1,3), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [28]:
one

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words=None, strip_accents='unicode', sublinear_tf=1,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f912685d840>, use_idf=1,
        vocabulary=None)

In [29]:
trn_term_doc = one.fit_transform(hate_df)

In [30]:
trn_term_doc

<800000x916218 sparse matrix of type '<class 'numpy.float64'>'
	with 29898162 stored elements in Compressed Sparse Row format>

In [31]:
indices_hate = np.argsort(one.idf_)[::-1]
features_hate = one.get_feature_names()
top_n = 100
top_features_hate = [features_hate[i] for i in indices_hate[:top_n]]
print(top_features_hate)

['50 bucks .', 'guilty one', 'bloody exams', 'social networking is', 'chris brown is', 'chris but', 'chris crocker', 'and i honestly', 'social security card', 'my brother (', 'bloody exam', 'social stream', '@ davidkyle', 'social studies .', 'social studies exam', 'social studies project', 'social this', 'social networking .', 'social networking ,', '@ davidleibrandt i', '@ davidmacmillan', 'social and', 'my brother but', 'social but', 'my brother broke', 'and i havn', 'social interaction', 'in 7 and', 'and i haz', '@ davidm _', 'social life is', '@ davidm', 'bloody foot', 'social media experts', 'social media tv', 'in 72', 'socialcalamity', 'socialise', 'society of', 'my broom', 'my broken leg', 'my broken laptop', '@ daviddevious', 'my broken camera', 'in 7th place', 'society today', '@ davidgilray', 'societyreject', '@ davidcook1', 'sociology .', 'sociology class', '@ davidclowney', 'sociology test', '@ davidgallant', 'societie ran out', 'chris farley .', 'guitar hero arcade', 'guit

### SAVING TOP WORDS HATE DATASET 

In [32]:
indices_hate = np.argsort(one.idf_)[::-1]
features_hate = one.get_feature_names()
top_n = 5000
top_features_hate = [features_hate[i] for i in indices_hate[:top_n]]

In [33]:
with open('z_hate_five', 'wb') as fp:
    pickle.dump(top_features_hate, fp)

## RETRIVING TOP WORDS FROM UNHATE DATASET 

In [34]:
data_unhate_five.head()

,tweet,class
800000,I LOVE @Health4UandPets u guys r the best!!,4
800001,im meeting up with one of my besties tonight! ...,4
800002,"@DaRealSunisaKim Thanks for the Twitter add, S...",4
800003,Being sick can be really cheap when it hurts t...,4
800004,@LovesBrooklyn2 he has that effect on everyone,4


In [35]:
unhate_df = data_unhate_five['tweet']
unhate_df.head()

800000         I LOVE @Health4UandPets u guys r the best!! 
800001    im meeting up with one of my besties tonight! ...
800002    @DaRealSunisaKim Thanks for the Twitter add, S...
800003    Being sick can be really cheap when it hurts t...
800004      @LovesBrooklyn2 he has that effect on everyone 
Name: tweet, dtype: object

In [36]:
trn_term_doc = one.fit_transform(unhate_df)

In [37]:
trn_term_doc

<800000x918970 sparse matrix of type '<class 'numpy.float64'>'
	with 28568155 stored elements in Compressed Sparse Row format>

In [38]:
indices_unhate = np.argsort(one.idf_)[::-1]
features_unhate = one.get_feature_names()
top_n = 100
top_features_unhate = [features_unhate[i] for i in indices_unhate[:top_n]]
print(top_features_unhate)

['informed by @', 'pocketedwardsev', 'by replying to', 'pocket _ poptart', 'by relient k', 'pocket edward', 'pocket for a', 'by relient', 'by really', 'by ray', 'pocketful', 'by proxy .', 'by r', 'by quickly !', 'pockets . .', 'pockets and', 'pockets are', 'pocketsmith', 'pocketsmith ,', 'pocketsmith , a', 'by richard', 'pocket . i', 'pocket . .', 'by rihanna', 'by seeing', 'by screaming', 'po in', 'po kayo', 'po ng', 'by school', "po po '", 'by saying i', 'poaching', 'poachontasbyn', 'poachontasbyn :', 'by say', 'pob34 hey', 'pocahontis', 'by sara', 'by robin', 'pocket -', 'pockettwit', 'pocky .', 'by the amount', 'podcasts for', 'by p !', 'by owl city', 'podcast that', 'podcast there', 'by owl', 'by outkast', 'by only', 'podcasts ?', 'podcasts while', 'by posting', 'podchat', 'podculture', 'poder', 'by old school', 'by old', 'podiobooks', 'by october', 'by now that', 'by paolo', 'podcast i', 'by paramore !', 'by paramore .', 'pocono 500', 'by popular demand', 'by popular', 'by plane'

### SAVING THE TOP WORDS FROM UNHATE DATASET 

In [39]:
indices_unhate = np.argsort(one.idf_)[::-1]
features_unhate = one.get_feature_names()
top_n = 5000
top_features_unhate = [features_unhate[i] for i in indices_unhate[:top_n]]

In [40]:
with open('z_unhate_five', 'wb') as fp:
    pickle.dump(top_features_unhate, fp)